<a href="https://colab.research.google.com/github/r-karra/ML4Sci-QMLHEP-GSoC2026-Evaluation/blob/main/circuit_implementations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task I: Quantum Computing Circuits

Implementation of two quantum circuits using PennyLane framework.

## Part 1: Multi-qubit Circuit with Hadamard, CNOT, SWAP, and Rotation
## Part 2: Swap Test Circuit

## Setup and Imports

In [ ]:
!pip install pennylane
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

print("PennyLane version:", qml.__version__)

# Part 1: Circuit 1 - Multi-Qubit Operations

**Requirements:**
- 5 qubits
- Hadamard gate on every qubit
- CNOT gates: (0,1), (1,2), (2,3), (3,4)
- SWAP on qubits (0,4)
- RX(π/2) rotation on any qubit

In [ ]:
# Define a device with 5 qubits
dev1 = qml.device('default.qubit', wires=5)

@qml.qnode(dev1)
def circuit_1():
    """
    Circuit 1: Multi-qubit gates demonstration

    Operations in order:
    1. Hadamard gates on all 5 qubits
    2. CNOT chain: (0,1), (1,2), (2,3), (3,4)
    3. SWAP operation between qubits 0 and 4
    4. RX(π/2) rotation on qubit 2
    """

    # Step 1: Apply Hadamard to all qubits
    for qubit in range(5):
        qml.Hadamard(wires=qubit)

    # Step 2: Apply CNOT chain
    cnot_pairs = [(0, 1), (1, 2), (2, 3), (3, 4)]
    for control, target in cnot_pairs:
        qml.CNOT(wires=[control, target])

    # Step 3: Apply SWAP between qubits 0 and 4
    qml.SWAP(wires=[0, 4])

    # Step 4: Apply RX(π/2) on qubit 2
    qml.RX(np.pi/2, wires=2)

    # Return expectation values (Z measurement on all qubits)
    return [qml.expval(qml.PauliZ(i)) for i in range(5)]

# Run the circuit
result1 = circuit_1()
print("\n=== CIRCUIT 1 RESULTS ===")
print(f"Expectation values of Z on all qubits: {result1}")

## Visualization of Circuit 1

In [ ]:
# Create a circuit drawer representation
print("\n=== CIRCUIT 1 DIAGRAM ===")
print(qml.draw(circuit_1)())

## Detailed Circuit Visualization with Matplotlib

In [ ]:
def visualize_circuit_1():
    """
    Visual representation of Circuit 1 operations
    """
    fig, ax = plt.subplots(figsize=(14, 8))

    # Define gate positions
    gate_sequence = [
        "Hadamard (0-4)",
        "CNOT (0→1)",
        "CNOT (1→2)",
        "CNOT (2→3)",
        "CNOT (3→4)",
        "SWAP (0↔4)",
        "RX(π/2) on qubit 2"
    ]

    # Plot circuit structure
    y_positions = np.arange(len(gate_sequence))
    ax.barh(y_positions, [1]*len(gate_sequence), color='skyblue', edgecolor='navy')
    ax.set_yticks(y_positions)
    ax.set_yticklabels(gate_sequence)
    ax.set_xlabel('Gate Sequence', fontsize=12)
    ax.set_title('Circuit 1: Multi-Qubit Gate Sequence', fontsize=14, fontweight='bold')
    ax.set_xlim(0, 1.2)

    # Add text annotations
    for i, (y, gate) in enumerate(zip(y_positions, gate_sequence)):
        ax.text(0.5, y, gate, ha='center', va='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig('circuit_1_visualization.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Circuit 1 visualization saved as 'circuit_1_visualization.png'")

visualize_circuit_1()

# Part 2: Circuit 2 - Swap Test

**Requirements:**
- Hadamard gate on qubit 0
- RX(π/3) rotation on qubit 1
- Hadamard gates on qubits 2 and 3
- Swap test between states |q0 q1⟩ and |q2 q3⟩

**Note:** Swap test is used to measure the overlap between two quantum states.

In [ ]:
# Define a device with 5 qubits (0-4, with qubit 4 as ancilla for swap test)
dev2 = qml.device('default.qubit', wires=5)

@qml.qnode(dev2)
def circuit_2():
    """
    Circuit 2: Swap Test

    Measures the overlap/similarity between two quantum states:
    - State 1: qubits 0 and 1
    - State 2: qubits 2 and 3
    - Ancilla qubit: qubit 4
    """

    # Step 1: Prepare first state on qubits 0-1
    qml.Hadamard(wires=0)          # H on qubit 0
    qml.RX(np.pi/3, wires=1)       # RX(π/3) on qubit 1

    # Step 2: Prepare second state on qubits 2-3
    qml.Hadamard(wires=2)          # H on qubit 2
    qml.Hadamard(wires=3)          # H on qubit 3

    # Step 3: Initialize ancilla qubit in superposition
    qml.Hadamard(wires=4)          # Ancilla in |+⟩

    # Step 4: Controlled-SWAP operations
    # CSWAP(control=4, target1=0, target2=2)
    qml.CSWAP(wires=[4, 0, 2])
    # CSWAP(control=4, target1=1, target2=3)
    qml.CSWAP(wires=[4, 1, 3])

    # Step 5: Final Hadamard on ancilla
    qml.Hadamard(wires=4)

    # Return measurements
    return qml.expval(qml.PauliZ(4))

# Run the circuit
result2 = circuit_2()
print("\n=== CIRCUIT 2 RESULTS ===")
print(f"Swap test ancilla measurement (Z expectation): {result2:.4f}")
print(f"Overlap between states (interpretation): {(1 - result2)/2:.4f}")

## Visualization of Circuit 2

In [ ]:
print("\n=== CIRCUIT 2 DIAGRAM ===")
print(qml.draw(circuit_2)())

## Detailed Swap Test Explanation

In [ ]:
def visualize_swap_test():
    """
    Visual explanation of the Swap Test algorithm
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Left panel: Circuit steps
    ax = axes[0]
    steps = [
        "Setup State 1\n(q0: H, q1: RX(π/3))",
        "Setup State 2\n(q2: H, q3: H)",
        "Ancilla Prep\n(q4: H)",
        "CSWAP(4, 0, 2)",
        "CSWAP(4, 1, 3)",
        "Final H(q4)",
        "Measure q4"
    ]

    y_pos = np.arange(len(steps))
    ax.barh(y_pos, [1]*len(steps), color='lightgreen', edgecolor='darkgreen')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(steps, fontsize=10)
    ax.set_xlabel('Step', fontsize=11)
    ax.set_title('Swap Test Circuit Steps', fontsize=12, fontweight='bold')
    ax.set_xlim(0, 1.2)

    # Right panel: Interpretation
    ax = axes[1]
    interpretations = [
        "Probability of measuring 0: (1 + overlap)/2",
        "Probability of measuring 1: (1 - overlap)/2",
        "",
        "Where overlap = |⟨ψ₁|ψ₂⟩|²",
        "",
        "Overlap = 0: Orthogonal states",
        "Overlap = 1: Identical states"
    ]

    ax.text(0.5, 0.95, 'Swap Test Interpretation', ha='center', va='top',
            fontsize=12, fontweight='bold', transform=ax.transAxes)

    y_start = 0.85
    for i, text in enumerate(interpretations):
        y = y_start - i*0.12
        if text:
            fontweight = 'bold' if 'Probability' in text or 'overlap' in text else 'normal'
            ax.text(0.05, y, text, fontsize=10, fontweight=fontweight,
                   transform=ax.transAxes, family='monospace')

    ax.axis('off')

    plt.tight_layout()
    plt.savefig('circuit_2_swap_test.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Swap test visualization saved as 'circuit_2_swap_test.png'")

visualize_swap_test()

## Comparison and Analysis

In [ ]:
print("\n" + "="*60)
print("CIRCUIT COMPARISON AND ANALYSIS")
print("="*60)

print("\n--- CIRCUIT 1: Multi-Qubit Gates ---")
print(f"Number of qubits: 5")
print(f"Hadamard gates: 5 (on all qubits)")
print(f"CNOT gates: 4 (chain operation)")
print(f"SWAP gates: 1")
print(f"RX gates: 1 (rotation angle: π/2)")
print(f"Total gate count: 11 gates")
print(f"Purpose: Demonstrates fundamental quantum operations and entanglement")

print("\n--- CIRCUIT 2: Swap Test ---")
print(f"Number of qubits: 5 (2 for state 1, 2 for state 2, 1 ancilla)")
print(f"Hadamard gates: 4 (state prep) + 2 (ancilla)")
print(f"RX gates: 1 (rotation angle: π/3)")
print(f"CSWAP (Fredkin) gates: 2")
print(f"Purpose: Measures overlap between two quantum states")
print(f"Overlap result: {(1 - result2)/2:.4f}")

print("\n--- KEY QUANTUM CONCEPTS DEMONSTRATED ---")
print("1. Superposition: Hadamard gates create equal superposition")
print("2. Entanglement: CNOT chain creates Bell-like states")
print("3. State Swapping: SWAP and CSWAP operations")
print("4. Rotation: RX gates rotate around X-axis")
print("5. Measurement: Z-basis measurements on qubits")
print("6. Swap Test: Computes overlap between two quantum states")